### Lagrangian v.s. Eulerian: Two Views of Continuums
1. Lagrangian View, 拉格朗日视角:
    - Sensors that move passively with the simulated material(随波逐流)
    - 粒子
2. Eulerian View, 欧拉视角:
    - Still sensors that never moves.(岿然不动)
    - 网格

### Mass-Spring System, 弹簧-质点模型
- Extremely ordinary
- But very useful!
    - Cloth
    - Elastic objects
    - ...

### Mathematical Model
\begin{equation*}
\begin{array}{rll}
    {\rm f}_ {ij} &=-k(||{\rm x}_ {i}-{\rm x}_ {j}||_ {2}- l_{ij})&(\widehat{{\rm x}_ {i}-{\rm x}_ {j}})~~~~(Hooke's Law) \\
    {\rm f}_ {i} &= \displaystyle\sum_{j}^{j\ne i}{\rm f}_ {ij}  & \\
    \displaystyle\frac{\partial {\rm v}}{\partial t} &= \displaystyle\frac{1}{m_{i}}{\rm f}_ {i} &
\end{array}
\end{equation*}

$k$: spring stiffness;

$l_{ij}$: spring rest length between particle $i$ and particle $j$;

$m_{i}$: the mass of particle $i$.


$(\widehat{{\rm x}_ {i}-{\rm x}_ {j}})$: direction vector from particle $i$ to particle $j$;

### Time integration
- Forward Euler (explicit)

\begin{equation*}
\begin{array}{rl}
    {\rm v}_ {t+1} &={\rm v}_ {t}+\Delta t\displaystyle\frac{{\rm f}_{t}}{m} \\
    {\rm x}_{t+1} &={\rm x}_ {t}+\Delta t{\rm v}_{t}
\end{array}
\end{equation*}

- Semi-implicit Euler (aka. sympletic Euler, explicit)

\begin{equation*}
\begin{array}{rl}
    {\rm v}_ {t+1} &= {\rm v}_ {t}+\Delta t\displaystyle\frac{{\rm f}_{t}}{m} \\
    {\rm x}_ {t+1} &= {\rm x}_ {t}+\Delta t{\rm v}_{t_1}
\end{array}
\end{equation*}

- Backward Euler (often with Newton's method, implicit)

#### Implementing a mass-spring system with sympletic Euler
Steps:
1. Compute new velocity using ${\rm v} {t+1}={\rm v}_ {t}+ \Delta t \frac{{\rm f}_{t}}{m}$
2. Collision with ground
3. Compute new position using ${\rm x}_ {t+1}={\rm x}_ {t}+ \Delta t {\rm v}_ {t_1}$

In [1]:
# Showcase
# Tutorials (Chinese):
# - https://www.bilibili.com/video/BV1UK4y177iH
# - https://www.bilibili.com/video/BV1DK411A771

import taichi as ti

ti.init(arch=ti.gpu)

spring_Y = ti.field(dtype=ti.f32, shape=())  # Young's modulus
paused = ti.field(dtype=ti.i32, shape=())
drag_damping = ti.field(dtype=ti.f32, shape=())
dashpot_damping = ti.field(dtype=ti.f32, shape=())

max_num_particles = 1024
particle_mass = 1.0
dt = 1e-3
substeps = 10

num_particles = ti.field(dtype=ti.i32, shape=())
x = ti.Vector.field(2, dtype=ti.f32, shape=max_num_particles)
v = ti.Vector.field(2, dtype=ti.f32, shape=max_num_particles)
f = ti.Vector.field(2, dtype=ti.f32, shape=max_num_particles)
fixed = ti.field(dtype=ti.i32, shape=max_num_particles)

# rest_length[i, j] == 0 means i and j are NOT connected
rest_length = ti.field(dtype=ti.f32,
                       shape=(max_num_particles, max_num_particles))


@ti.kernel
def substep():
    n = num_particles[None]

    # Compute force
    for i in range(n):
        # Gravity
        f[i] = ti.Vector([0, -9.8]) * particle_mass
        for j in range(n):
            if rest_length[i, j] != 0:
                x_ij = x[i] - x[j]
                d = x_ij.normalized()

                # Spring force
                f[i] += -spring_Y[None] * (x_ij.norm() / rest_length[i, j] -
                                           1) * d

                # Dashpot damping
                v_rel = (v[i] - v[j]).dot(d)
                f[i] += -dashpot_damping[None] * v_rel * d

    # We use a semi-implicit Euler (aka symplectic Euler) time integrator
    for i in range(n):
        if not fixed[i]:
            v[i] += dt * f[i] / particle_mass
            v[i] *= ti.exp(-dt * drag_damping[None])  # Drag damping

            x[i] += v[i] * dt
        else:
            v[i] = ti.Vector([0, 0])

        # Collide with four walls
        for d in ti.static(range(2)):
            # d = 0: treating X (horizontal) component
            # d = 1: treating Y (vertical) component

            if x[i][d] < 0:  # Bottom and left
                x[i][d] = 0  # move particle inside
                v[i][d] = 0  # stop it from moving further

            if x[i][d] > 1:  # Top and right
                x[i][d] = 1  # move particle inside
                v[i][d] = 0  # stop it from moving further


@ti.kernel
def new_particle(pos_x: ti.f32, pos_y: ti.f32, fixed_: ti.i32):
    # Taichi doesn't support using vectors as kernel arguments yet, so we pass scalars
    new_particle_id = num_particles[None]
    x[new_particle_id] = [pos_x, pos_y]
    v[new_particle_id] = [0, 0]
    fixed[new_particle_id] = fixed_
    num_particles[None] += 1

    # Connect with existing particles
    for i in range(new_particle_id):
        dist = (x[new_particle_id] - x[i]).norm()
        connection_radius = 0.15
        if dist < connection_radius:
            # Connect the new particle with particle i
            rest_length[i, new_particle_id] = 0.1
            rest_length[new_particle_id, i] = 0.1


@ti.kernel
def attract(pos_x: ti.f32, pos_y: ti.f32):
    for i in range(num_particles[None]):
        p = ti.Vector([pos_x, pos_y])
        v[i] += -dt * substeps * (x[i] - p) * 100


def main():
    gui = ti.GUI('Explicit Mass Spring System',
                 res=(512, 512),
                 background_color=0xDDDDDD)

    spring_Y[None] = 1000
    drag_damping[None] = 1
    dashpot_damping[None] = 100

    new_particle(0.3, 0.3, False)
    new_particle(0.3, 0.4, False)
    new_particle(0.4, 0.4, False)

    while True:
        for e in gui.get_events(ti.GUI.PRESS):
            if e.key in [ti.GUI.ESCAPE, ti.GUI.EXIT]:
                exit()
            elif e.key == gui.SPACE:
                paused[None] = not paused[None]
            elif e.key == ti.GUI.LMB:
                new_particle(e.pos[0], e.pos[1],
                             int(gui.is_pressed(ti.GUI.SHIFT)))
            elif e.key == 'c':
                num_particles[None] = 0
                rest_length.fill(0)
            elif e.key == 'y':
                if gui.is_pressed('Shift'):
                    spring_Y[None] /= 1.1
                else:
                    spring_Y[None] *= 1.1
            elif e.key == 'd':
                if gui.is_pressed('Shift'):
                    drag_damping[None] /= 1.1
                else:
                    drag_damping[None] *= 1.1
            elif e.key == 'x':
                if gui.is_pressed('Shift'):
                    dashpot_damping[None] /= 1.1
                else:
                    dashpot_damping[None] *= 1.1

        if gui.is_pressed(ti.GUI.RMB):
            cursor_pos = gui.get_cursor_pos()
            attract(cursor_pos[0], cursor_pos[1])

        if not paused[None]:
            for step in range(substeps):
                substep()

        X = x.to_numpy()
        n = num_particles[None]

        # Draw the springs
        for i in range(n):
            for j in range(i + 1, n):
                if rest_length[i, j] != 0:
                    gui.line(begin=X[i], end=X[j], radius=2, color=0x444444)

        # Draw the particles
        for i in range(n):
            c = 0xFF0000 if fixed[i] else 0x111111
            gui.circle(pos=X[i], color=c, radius=5)

        gui.text(
            content=
            f'Left click: add mass point (with shift to fix); Right click: attract',
            pos=(0, 0.99),
            color=0x0)
        gui.text(content=f'C: clear all; Space: pause',
                 pos=(0, 0.95),
                 color=0x0)
        gui.text(content=f'Y: Spring Young\'s modulus {spring_Y[None]:.1f}',
                 pos=(0, 0.9),
                 color=0x0)
        gui.text(content=f'D: Drag damping {drag_damping[None]:.2f}',
                 pos=(0, 0.85),
                 color=0x0)
        gui.text(content=f'X: Dashpot damping {dashpot_damping[None]:.2f}',
                 pos=(0, 0.8),
                 color=0x0)
        gui.show()


if __name__ == '__main__':
    main()

[Taichi] mode=release
[Taichi] version 0.7.20, llvm 10.0.0, commit 284f75ed, win, python 3.8.10
[Taichi] Starting on arch=cuda
[Taichi] materializing...


RuntimeError: Window close button clicked, exiting... (use `while gui.running` to exit gracefully)

### 雅可比迭代法
对于矩阵 $Ax=b$, $A$非奇异，且对角元不为0，可以将原方程组改写为：

\begin{equation*}
\left\{  
             \begin{array}{**lr**}  
             x_{1}=\displaystyle\frac{1}{a_{11}}\left(b_{1}-a_{11}x_{2}-...-a_{1n}x_{n}\right), &  \\  
             x_{2}=\displaystyle\frac{1}{a_{22}}\left(b_{2}-a_{21}x_{1}-...-a_{2n}x_{n}\right), & \\ 
             ...... & \\ 
             x_{n}=\displaystyle\frac{1}{a_{nn}}\left(b_{n}-a_{n1}x_{1}-...-a_{(n,n-1)}x_{n-1}\right), &   
             \end{array}  
\right.
\end{equation*}

In [2]:
import taichi as ti
import random

ti.init(arch=ti.cpu)

n = 20

A = ti.field(dtype=ti.f32, shape=(n, n))
x = ti.field(dtype=ti.f32, shape=n)
new_x = ti.field(dtype=ti.f32, shape=n)
b = ti.field(dtype=ti.f32, shape=n)


# 单步雅可比迭代
@ti.kernel
def iterate():
    for i in range(n):
        r = b[i]
        for j in range(n):
            if i != j:
                r -= A[i, j] * x[j]

        new_x[i] = r / A[i, i]

    for i in range(n):
        x[i] = new_x[i]


# 计算误差
@ti.kernel
def residual() -> ti.f32:
    res = 0.0

    for i in range(n):
        r = b[i] * 1.0
        for j in range(n):
            r -= A[i, j] * x[j]
        res += r * r

    return res


for i in range(n):
    for j in range(n):
        A[i, j] = random.random() - 0.5

    A[i, i] += n * 0.1

    b[i] = random.random() * 100

for i in range(100):
    iterate()
    print(f'{i}, residual={residual():0.10f}')

for i in range(n):
    lhs = 0.0
    for j in range(n):
        lhs += A[i, j] * x[j]
    assert abs(lhs - b[i]) < 1e-4

[Taichi] Starting on arch=x64
[Taichi] materializing...
0, residual=24736.9296875000
1, residual=3674.2048339844
2, residual=1047.6331787109
3, residual=479.8469848633
4, residual=127.7566528320
5, residual=38.9668617249
6, residual=14.7183237076
7, residual=7.2304415703
8, residual=3.6475548744
9, residual=2.1913433075
10, residual=0.9774737954
11, residual=0.5886521935
12, residual=0.2952284813
13, residual=0.1701266766
14, residual=0.0896255150
15, residual=0.0499186404
16, residual=0.0264424756
17, residual=0.0149449417
18, residual=0.0079610366
19, residual=0.0044234050
20, residual=0.0023861893
21, residual=0.0013163054
22, residual=0.0007145723
23, residual=0.0003932702
24, residual=0.0002135455
25, residual=0.0001171976
26, residual=0.0000639685
27, residual=0.0000350286
28, residual=0.0000190337
29, residual=0.0000104412
30, residual=0.0000057530
31, residual=0.0000031255
32, residual=0.0000016539
33, residual=0.0000009213
34, residual=0.0000005376
35, residual=0.0000002782
36

In [2]:
import taichi as ti

ti.init(arch=ti.gpu)

a = ti.field(dtype=ti.f32, shape=1)

print("hello world")


[Taichi] Starting on arch=cuda
hello world


$$ y = a + b x $$